In [ ]:
from datetime import datetime, timedelta
from pandas import DataFrame
from json import dumps

from maystreet_data.api_client import ApiClient, ProvidedApiClientOptions

# -- create an API client

client = ApiClient()


# -- how to get a list of all supported calls

all_functions = client.get_functions()
all_functions

In [ ]:
# -- how to get information about a particular call

?client.mrn.get_news_by_ric

In [ ]:
# -- how to call the Foundation API (MayStreet Data Lake)

dt = '2020-01-02'
product = 'AAPL'
f = 'bats_edga'
columns = ['f', 'dt', 'quantity']

output_results = DataFrame(list(client.foundation.get_mt_trade(dt=dt, product=product, f=f, columns=columns)))
output_results

In [ ]:
# -- how to call a RTH function to create minute bars

bars = DataFrame(
    client.rth.create_minute_bars(
        start_date='2020-11-23 00:00:00.000000',
        finish_date='2020-11-23 23:59:59.999999',
        bar_size_in_seconds=3600,
    )
)

bars

In [ ]:
# -- how to call a RTH function to get news sentiment data by a RIC

start_date = '2019-01-08 12:00:11.000000'
finish_date = '2019-01-08 13:00:11.000000'
ric = 'IBM.DE'

ibm_news_sentiments = DataFrame(list(client.mrn.get_news_sentiment_by_ric(start_date=start_date, finish_date=finish_date, ric=ric)))
ibm_news_sentiments

In [ ]:
# -- how to run a pure SQL query; note that each of the backend provides ('foundation', rth, and mrn) all have an 'exec' function which allows SQL execution.

import maystreet_data

records = DataFrame(list(client.rth.exec(sql="""
    SELECT
        Date_Time,
        RIC,
        Bid_Price
    FROM
        `dbd-sdlc-prod.LSE_NORMALISED_5Y`.LSE_NORMALISED
    WHERE
        Date_Time BETWEEN '2020-11-23 15:00:00.000000' AND '2020-11-23 15:01:00.000000'
        AND Bid_Price > 0
    ORDER BY
        Date_Time
""",
)))

records

In [ ]:
# -- a larger example of getting news sentiments, and then getting data around the time of the news event

start_date = '2019-01-08 12:00:11.000000'
finish_date = '2019-01-08 13:00:11.000000'
ric = 'IBM.DE'

ibm_news_sentiments = list(
    client.mrn.get_news_sentiment_by_ric(start_date=start_date, finish_date=finish_date, ric=ric))

columns = ['ric', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'date_time', 'date_time_nano_fraction']

for row in ibm_news_sentiments:
    news_date = row['timestamp']
    news_headline = row['news_headline']

    target_date = datetime.strptime(news_date, "%Y-%m-%dT%H:%M:%S.%fZ")
    news_start_date = target_date - timedelta(days=0, hours=0, minutes=5)
    news_finish_date = target_date + timedelta(days=0, hours=0, minutes=5)

    notified_row = False

    print(
        f"'{news_headline}' @ '{news_date} | sentiment: positive: {row['sentiment_positive']}, negative: {row['sentiment_negative']}, neutral: {row['sentiment_neutral']}")

    trades_data = list(client.rth.get_ger_normalised_data(start_date=news_start_date, finish_date=news_finish_date,
                                                          ric=ric,
                                                          distinct=0,
                                                          use_like_for_ids=False, columns=columns))

    trades_data.sort(key=lambda r: r['date_time'])

    for trade_row in trades_data:
        trade_row_date = datetime.strptime(trade_row['date_time'], "%Y-%m-%dT%H:%M:%S.%fZ")
        print(
            f"time: {trade_row['date_time']} | ask vol: {trade_row['ask_size']}, ask px: {trade_row['ask_price']}, bid vol: {trade_row['bid_size']}, bid px: {trade_row['bid_price']},  ")

        if trade_row_date > target_date and not notified_row:
            notified_row = True
            print(' --- NEWS EVENT ---')

    print('----------------------------------------------------------------------------------------------------')
